#  Data Analysis and Results Collection

This file is for automaticly output all the analysis for all dataset at once 

In [ ]:
#!pip install xlwt

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
#Imports
import math
import librosa
import librosa.display
import IPython.display as ipd

import pyAudioAnalysis
import pydub
import ffmpeg
import os
import sklearn

from pydub import AudioSegment
from pydub.silence import split_on_silence

import numpy
import numpy as np
import matplotlib.pyplot as plt

import xlwt
from xlwt import Workbook

from numpy import sin, cos, pi, linspace, arange, log10, absolute
from numpy.random import randn
from scipy.signal import lfilter, lfilter_zi, filtfilt, butter, freqz, welch
from scipy.io.wavfile import read , write
from scipy.signal import spectrogram
from matplotlib import pyplot
from matplotlib.pyplot import ylabel, xlabel
from matplotlib.pyplot import plot, legend, show, grid, figure, savefig#, hold

In [ ]:
#Cepstrum and Power Spectrum

def real_cepstrum(x, n=None):
# Compute the real cepstrum of a real sequence
#    x : ndarray
#        Real sequence to compute real cepstrum of.
#    n : {None, int}, optional
#        Length of the Fourier transform.
#    Returns
#    -------
#    ceps: ndarray
#        The real cepstrum.

    spectrum = np.abs(np.fft.fft(x, n=n))**2
    ceps = np.fft.ifft(np.log(spectrum))

    return ceps, spectrum`

In [ ]:
#Zero Crossing Rate

def ZCR(samples, frameSize, overlap):
    wlen = len(samples)
    step = frameSize - overlap
    frameNum = math.ceil(wlen/step)
    zcr = np.zeros((frameNum,1))
    for i in range(frameNum):
        curFrame = samples[np.arange(i*step,min(i*step+frameSize,wlen))]
        #To avoid DC bias, usually we need to perform mean substraction on each frame
        curFrame = curFrame - np.mean(curFrame) #Zero-Justified
        zcr[i] = sum(curFrame[0:-1]*curFrame[1::]<=0)
    return zcr

In [ ]:
def spectral_centroid(x, samplerate=44100):
    magnitudes = np.abs(np.fft.rfft(x)) # magnitudes of positive frequencies
    length = len(x)
    freqs = np.abs(np.fft.fftfreq(length, 1.0/samplerate)[:length//2+1]) # positive frequencies
    return np.sum(magnitudes*freqs) / np.sum(magnitudes) # return weighted mean

In [32]:
#List of Words
words = ["chata","chapa","chave","lata","lapa","lava","casa","capa","cave","cata","chuta","chupa","chuva","farta","farpa","farda","ripa","rita","rica"]

#List of Dataset versions
#d = {"one": 1, "two": 2, "three": 3, "Four": 4, "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10}

d = {"one": 1, "two": 2, "three": 3, "Four": 4, "five": 5, "six": 6, "seven": 7, "eight": 8,
     "nine": 9, "ten": 10,  "eleven": 11, "twelve": 12, "thirteen": 13, "fourteen": 14, "fifteen": 15,
     "sixteen": 16, "seventeen": 17, "eighteen": 18, "nineteen": 19, "twenty": 20}

iterable = d.keys()

#List of people
p = ['A','J','N']

#List of Phonemes 
phoneme = ["_0", "_1"]

In [5]:
segments = []
errors = 0

# Workbook is created
wb = Workbook()

# add_sheet is used to create sheet.
sheet1 = wb.add_sheet('Word Analysis')

sheet1.write(1, 1, 'Max PW_spec')
sheet1.write(2, 1, 'Ceps')
sheet1.write(3, 1, 'Avg ZCR')
sheet1.write(4, 1, 'Spec_zen')

w = 0
h = 1

for y in words:
    for item in iterable:
        for x in p:
            for t in phoneme:
                
                if (w == 9):
                    w = 0
                h += 1
                # Write file path
                l = str(d[item])
                os.chdir(r'C:\Users\Dasil\1. Processamento Digital do Sinal\Project')
                
                #MFCC Analysis
                filename = 'audio/WD_data/'+x+'_'+y+'_0'+l+t+'.wav' #for Test DataSet
                signal, fs = librosa.load(filename)
                
                sr = fs
                
                #signal.shape
                mfccs = librosa.feature.mfcc(signal, sr=fs, n_mfcc=40, n_fft = 1024, hop_length = 50, n_mels = 130,  fmin = 10, fmax = 4000)         
                #mfccs = librosa.feature.mfcc(signal, sr=fs, n_mfcc=40, n_fft = 1024, hop_length = 200, n_mels = 1000,  fmin = 10, fmax = 4000)         

                #Normalyse Y scale
                mfccs = sklearn.preprocessing.scale(mfccs, axis=1)

                #Visualie MFCC 
                plt.figure(figsize=(14,4))
                #librosa.display.specshow(mfccs[1:][10:],x_axis='time',sr=sr)
                librosa.display.specshow(mfccs[25:][:],x_axis='time',sr=sr)
                plt.colorbar(format="%+2f")
                #plt.plot()
                #plt.savefig('Word Detection/MFCCS_ID/'+x+y+l+t+'.jpeg')
                
                #Calculate Cepstrung and Power Spectrum
                ceps, spec = real_cepstrum(signal, n=None)
                N = signal.shape[0]

                #Power Specturm
                power_spec = np.abs(spec[:N//2])**2
                #plt.plot(power_spec[0:1500])
                index = np.where(power_spec == np.max(power_spec))
                print(f'Power_spec: {index[0][0]}')
                mfccs.append(index[0][0])

                #Calculate Zero Cross Rate
                frameSize = 256
                overlap = 0
                zcr = ZCR(signal, frameSize, overlap)
                print(f'Zero Cross Rate: {np.max(zcr)}')
                mfccs.append(np.max(zcr))

                #Calculate Cepstrum
                abs_ceps = np.abs(ceps[:N//2])**2
                #plt.plot(abs_ceps[:10])
                print(f'Cepstrum: {round(np.max(abs_ceps))}')
                
                #Spectral Centroid
                spec_cen = spectral_centroid(signal, fs)
                print(f'Spectral Centroid: {round(spec_cen)}')
                
                w +=1
                sheet1.write(w, h, int(index[0][0]))
                w +=1
                sheet1.write(w, h, round(np.max(abs_ceps)))
                w +=1 
                sheet1.write(w, h, round(np.average(zcr)))
                w +=1
                sheet1.write(w, h, round(spec_cen))  
                
                
                #Clean the graphics
                plt.close()
                plt.cla()
                plt.clf()
                    

<Figure size 432x288 with 0 Axes>

In [33]:
for y in words:
    for item in iterable:
        for x in p:
                # Write file path
                if(d[item] <10):
                    k = '0'
                elif(d[item] == 10):
                    k=''
                elif(d[item] == 20):
                    k=''
                else:
                    k = '0' 
                l = str(d[item])
                os.chdir(r'C:\Users\Dasil\1. Processamento Digital do Sinal\Project')
                
                #MFCC Analysis
                filename = 'audio/PD_data/'+x+'_'+y+'_'+k+l+'_1.wav' #for Test DataSet
                
                l = d[item]
                
                # Read file path
                fs, audio1 = read(filename)
                
                # Fix-sized segmentation (breaks a signal into non-overlapping segments)
                signal = audio1 / (2**15)
                signal_len = len(signal)
                segment_size_t = 1 # segment size in seconds
                segment_size = segment_size_t * fs  # segment size in samples

                # Break signal into list of segments in a single-line Python code
                segment1 = audio1
                segments = [segment1]
                
                #l += 10
                
                l = str(l)
                
                if(d[item] <10):
                    k = '0'
                elif(d[item] > 10):
                    k=''
                
                for iS, s in enumerate(segments):
                        write('audio/PD_data2/'+x+'_'+y+'_'+k+l+'_1.wav'.format(segment_size_t * iS, segment_size_t * (iS + 1)), fs, (s))
                    
